# Deploy KanjuTech Transcription and Speaker Diarization Model Package from AWS Marketplace 




KanjuTech Transcription and Diarization model allows full end-to-end recognition of conversations with multiple participants, including cases with speech overlaps. The model utilizes GPU acceleration and allows the processing of a one-hour recording in less than 7.5 minutes. It supports 19 languages (WER<10%).

This sample notebook shows you how to deploy [**KanjuTech Transcription and Diarization Model**](https://aws.amazon.com/marketplace/pp/prodview-ngtdx4ayt4emo?sr=0-1&ref_=beagle&applicationId=AWSMPContessa) using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [**KanjuTech Transcription and Diarization Model**](https://aws.amazon.com/marketplace/pp/prodview-ngtdx4ayt4emo?sr=0-1&ref_=beagle&applicationId=AWSMPContessa). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    
The maximum audio file size for real-time inference is 20MB and for batch transform job is 100MB each file. The recommended duration of one audio file for real-time inference is limited to 7-8 minutes.

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [**KanjuTech Transcription and Diarization Model**](https://aws.amazon.com/marketplace/pp/prodview-ngtdx4ayt4emo?sr=0-1&ref_=beagle&applicationId=AWSMPContessa).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
model_package_arn = "<Specify Model package ARN corresponding to your AWS region>"

In [ ]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import numpy as np
import s3fs
import os

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

#bucket = sagemaker_session.default_bucket() # use this line if you want to create default S3 bucket
bucket = '<Name-of-your-existing-S3-bucket>' # write name of your S3 bucket where you store your input files and want to save the output
runtime = boto3.client("runtime.sagemaker")
bucket

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
model_name = "kanjutech-transcription-speaker-diarization"

real_time_content_type = "application/json"
batch_transform_content_type = "audio/wav"
accept = "application/json"

real_time_inference_instance_type = (
    "ml.g4dn.xlarge"
)
batch_transform_inference_instance_type = (
    "ml.p3.2xlarge"
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

Input audio files from S3 bucket and encode in base64 (for endpoints). For real-time inference local-stored files can be used. 
The maximum audio file size for real-time inference is 20MB each file. The recommended duration of one audio file for real-time inference is limited to 7-8 minutes.

In [ ]:
fs = s3fs.S3FileSystem()
fs_ls = fs.ls(bucket+'/'+'<Your folder on S3 bucket where you store input audio>')
paths = list(filter(lambda k: '.' in k, fs_ls))

# We encode only one file from paths for example
input_file_path = paths[0]

with fs.open(input_file_path, "rb") as f:
    edata = base64.b64encode(f.read())

Create json file with encoded data, and additional parameters:
'language': use "auto" for automatic language identification or "en": "english", "ca": "catalan", "nl": "dutch", "fi": "finnish", "fr": "french", "de": "german", "id": "indonesian", "it": "italian", "ja": "japanese", "ms": "malay", "no": "norwegian", "pl": "polish", "pt": "portuguese", "ru": "russian", "es": "spanish", "sv": "swedish", "tr": "turkish", "uk": "ukrainian", "vi": "vietnamese".
'num_speakers': use "auto" for automatic identification of speakers or write a specific number

In [ ]:
json_file = {}
json_file['file'] = edata.decode('utf-8')
json_file['language'] = "en"
json_file['num_speakers'] = 2
json_file['f_name'] = os.path.basename(input_file_path)
data = json.dumps(json_file)

### C. Perform real-time inference

Invoke the endpoint for real-time inference

In [ ]:
results = runtime.invoke_endpoint(
    EndpointName=model_name,
    Body=data, 
    ContentType=real_time_content_type,  
    Accept=accept,  
)

### D. Visualize output

In [ ]:
print(results['Body'].read().decode())

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

Upload input files to S3 for batch transform job. The maximum audio file size for batch transform job is 100MB each file.

In [ ]:
transform_input_folder = bucket+'/'+'<Your folder on S3 bucket where you store input audio>'
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

Or use your existing S3 bucket with input files

In [ ]:
transform_input = bucket+'/'+'<Your folder on S3 bucket where you store input audio>'

Run batch transform job

Specify language and number of speakers: 
"LANGUAGE": use "auto" for automatic language identification or "en": "english", "ca": "catalan", "nl": "dutch", "fi": "finnish", "fr": "french", "de": "german", "id": "indonesian", "it": "italian", "ja": "japanese", "ms": "malay", "no": "norwegian", "pl": "polish", "pt": "portuguese", "ru": "russian", "es": "spanish", "sv": "swedish", "tr": "turkish", "uk": "ukrainian", "vi": "vietnamese".
'NUM_SPEAKERS': use "auto" for automatic identification of speakers or write a specific number.

In [ ]:
transformer = model.transformer(1, batch_transform_inference_instance_type, 
                                output_path=bucket+'/'+'<Your folder on S3 bucket where you want to store output audio>', accept = "application/json",
                               max_payload=100, env={"LANGUAGE": "en", "NUM_SPEAKERS": "2"})
transformer.transform(transform_input, content_type=batch_transform_content_type)
transformer.wait()

Visualize output

In [ ]:
# output is available on following path
transformer.output_path

In [ ]:
fs = s3fs.S3FileSystem()

with fs.open(transformer.output_path+'/'+'<Name-of-the-audio-file>.wav.out', "r") as f:
    output = f.read()
output

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

